# Week 5 - NLP and Deep Learning

---

# Lecture 9: Sequence Prediction with HMMs

In this exercise you will implement the Viterbi algorithm for decoding in sequence tagging. More concretely, we are going to build a POS tagger for English web data.

## 1. Emissions and transition probabilities

In this part of the exercise you are going to prepare the emission and transition probabilities to use in the viterbi algorithm. We are going to focus on the task of Parts-Of-Speech (POS) tagging. You can use the following datareader for the following assignments:

In [ ]:
def read_conll_file(path):
    """
    read in conll file
    
    :param path: path to read from
    :returns: list with sequences of words and labels for each sentence
    """
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue # skip comments
            tok = line.split('\t')

            current_words.append(tok[0])
            current_tags.append(tok[1])
        else:
            if current_words:  # skip empty lines
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    # check for last one
    if current_tags != []:
        data.append((current_words, current_tags))
    return data

Because we are going to use the the POS labels as indices in our Viterbi matrix, we need to know all labels beforehand, and they need to have a static order. We also need a special beginning and end label:

In [ ]:
train_data = read_conll_file('pos-data/en_ewt-train.conll')
dev_data = read_conll_file('pos-data/en_ewt-dev.conll')

SMOOTH = 0.1
BEG = '<S>'
END = '</S>'
UNK = '<UNK>'

label_set = set([pos_label for sentence in train_data for pos_label in sentence[1]])
label_set.add(BEG)
label_set.add(END)
# put labels in a list, so that they are guaranteed to have the same order
label_list = list(sorted(label_set))

print('Length train data: ' + str(len(train_data)))
print('Length dev data: ' + str(len(dev_data)))

# the data is a list of pairs, containing 1: a list of words 2: a list of POS labels
print('Random datapoint:')
print(train_data[70][0])
print(train_data[70][1])

print('All labels:')
print(label_list)

* a) calculate the transition probabilities based on the training data, use a special label for the beginning of a sentence (`<S>`) and the end of a sentence (`</S>`). use laplace smoothing with a value of 0.1 to avoid probabilities of 0.0.

**Hint**: The transition probability $P(t_i|t_{i-1})$ is the probability that given a tag, $t_{i-1}$, that it will be followed by a tag $t_i$. 
$$P(t_i|t_{i-1}) = {C(t_{i-1},t_{i}) \over C(t_{i-1})}$$
With smoothing:
$$P(t_i|t_{i-1}) = {C(t_{i-1},t_{i}) + \gamma \over C(t_{i-1}) + (|t|) * \gamma} $$

Where $(|t|-1) * \gamma$ is used because we want to add probability mass to all labels.

**Hint2**: Every sentence in the data looks like `['DET', 'NOUN', 'VERB']` without any `<S>` or `</S>` tags. So the beginning and end of every data sample needs to be handled differently when counting occurences of transitions, alternatively you can add these tokens to each data sample so they look like `['<S>, 'DET', 'NOUN', 'VERB', </S>]`

In [ ]:
# Make a dictionary of dictionaries, so that we can easily query for a certain probability.
# We add the smoothing value as a starting point, as it has to be added to each count.
# Note that a list of lists with POS indices is more efficient (but a bit more cumbersome to implement).
transition_counts = {label: {label: SMOOTH for label in label_list} for label in label_list}
# The count that a NOUN follows an ADJ (empty now)
print(transition_counts['ADJ']['NOUN'])

# First obtain the raw counts
for sentence in train_data:
    labels = [BEG] + sentence[1] + [END]
    for pos_idx in range(1,len(labels)):
        prev = labels[pos_idx-1]
        cur = labels[pos_idx]
        transition_counts[prev][cur] += 1
        
print(transition_counts['ADJ']['NOUN']) # should be 6803.1

# Now fill the transition matrix, note that the outgoing probability of each label should sum to 1.0
transition_probs = {label: {label: 0.0 for label in label_list} for label in label_list}
for prev in label_list:
    total_prev = sum(transition_counts[prev].values())
    for cur in label_list:
        prob = transition_counts[prev][cur]/total_prev
        transition_probs[prev][cur] = prob

* b) calculate the emission probabilities based on the training data. Make sure that every POS tag can be assigned to an `<UNK>` token, use laplace smoothing with a value of 0.01 to avoid probabilities of 0.0.

**Hint**: The emission probability $P(w_i|t_{i})$ is the probability that given a tag, $t_i$, that it will be associated with a given word $w_i$. The formula below shows counts $C$ needed to calculate the probability.

$$P(w_i|t_{i}) = {C(t_{i},w_{i}) \over C(t_{i})}$$

In [ ]:
word_set = {UNK}
for sent in train_data:
    for word in sent[0]:
        word_set.add(word)
word_list = list(sorted(word_set))

emission_counts = {label: {word: SMOOTH for word in word_list} for label in label_list}

for sent in train_data:
    for word, label in zip(sent[0], sent[1]):
        emission_counts[label][word] += 1


emission_probs = {label: {word: SMOOTH for word in word_list} for label in label_list}
for label in label_list:
    total_label = sum(emission_counts[label].values())
    for word in word_list:
        emission_probs[label][word] = emission_counts[label][word]/total_label

import pickle
pickle.dump((transition_probs, emission_probs), open('probs_en.pickle', 'wb'))

You can check whether your solutions are correct by estimating the probabilities on the data and check whether the probabilities match:

In [ ]:
print(transition_probs['ADJ']['NOUN']) # 0.5171926196793345
print(transition_probs['NOUN']['ADJ']) # 0.01123434129302644
print(transition_probs[BEG]['ADJ']) # 0.04082136964025221
print(transition_probs['ADJ'][END]) # 0.003808756338424345
print(emission_probs['NOUN']['calling'])   # 2.9909103515414666e-05
print(emission_probs['VERB']['calling'])  # 0.0005740785225418476
print(emission_probs['VERB'][UNK])  # 4.071478883275515e-06

## 2. Viterbi algorithm

In the image below we see an example of the calculation of the first 2 positions in a Viterbi decoding:
<img width=500px src="pics/viterbi.jpg">

* a) Implement Viterbi decoding, use the transition and emission probabilities previously estimated (note that we also provide pre-calculated probabilities in `probs_en.pickle`). You can use the example code shown below as a starting point if you like.

**Hint**: The implementation can become simpler if you think about the problem as a 2d matrix that needs to be filled (each position in the list is a node in the viterbi decoding, $v_1(7)$, $v_1(6)$, ...). You can first initialize the matrix with 0.0's, and then fill it from left to right.

**Hint2**: You need to combine three probabilities for each possible history, and take the max of all possible histories. You do not need to use negative log probabilities for now.

In [ ]:
# You can also load the pre-calculate probabilities:
# import pickle
# transition_probs, emission_probs = pickle.load(open('probs_en.pickle', 'rb'))
import numpy as np
def viterbi(sentence):
    """
    sentence: list of strings
    """
    columns = len(sentence)
    # You don't need the special tokens in the viterbi decoding so we remove them
    labels_list_exc_special = label_list.copy()
    labels_list_exc_special.remove(BEG)
    labels_list_exc_special.remove(END)

    rows = len(labels_list_exc_special)
    
    # Create the full matrix for scores as well as the backtracking.
    # e.g. scores[0][3] should get the probability of the best path of 
    # the first label and the 4th word in the sentence
    # Backtrack contains the index of the best tag for the previous word
    # e.g. backtrack[0][3] should get the index of the best tag for the 3rd word 
    # when backtracking from the first label and 4th word
    scores = np.array([[0.0 for _ in range(columns)] for _ in range(rows)])
    backtrack = np.array([[0 for _ in range(columns)] for _ in range(rows)])
    
    # Handle the first token separately, as it only has 2 probabilities (emission, transition)
    word_position = 0
    for pos_tag_idx, pos_tag in enumerate(labels_list_exc_special):
        # The probability of the first word given the POS tag:
        word = sentence[word_position]
        if word not in emission_probs[pos_tag]:
            word = UNK
        em_prob = emission_probs[pos_tag][word] 
        
        # The probability of the POS tag given that the previous "tag" is <S>
        transition_prob = transition_probs[BEG][pos_tag]
        
        # Save the total probability:
        scores[pos_tag_idx][word_position] = em_prob * transition_prob
        
        # Backtracking for the first token is uneccessary so we ignore it
    
    # Now handle the rest of the sequence
    for word_position in range(1, columns):
        for pos_tag_idx, pos_tag in enumerate(labels_list_exc_special):

            # Get emission probability, remember to handle unknown words
            word = sentence[word_position]
            if word not in emission_probs[pos_tag]:
                word = UNK
            em_prob = emission_probs[pos_tag][word]
            
            # For each possible path
            # Get the transition probability and the history probability for each candidate tag
            # Hint: the history probability is the score of the previous word position in scores matrix
            # TODO
            candidate_scores = [0]*len(labels_list_exc_special)
            for prev_label_idx, prev_label in enumerate(labels_list_exc_special):
                history_prob = scores[prev_label_idx][word_position-1]
                transition_prob = transition_probs[prev_label][pos_tag]
                candidate_scores[prev_label_idx] = history_prob * transition_prob * em_prob
                
            # Now extract the best score from candidate_scores and its previous path and save these
            # Hint: backtrack should contain the index of the best tag backtrack[tag_idx][word_position] = previous_best_tag_idx
            # TODO
            most_prob = max(candidate_scores)
            best_history = candidate_scores.index(most_prob)
            backtrack[pos_tag_idx][word_position] = best_history
            scores[pos_tag_idx][word_position] = most_prob


    # Extract the best score from the last labels to the special end label
    # Hint: here you only have history and transition (no emission)
    # TODO
    candidate_scores = [0.0] * len(labels_list_exc_special)
    for prev_label_idx, prev_label in enumerate(labels_list_exc_special):
        history_prob = scores[prev_label_idx][-1]
        transition_prob = transition_probs[prev_label][END]
        score = history_prob * transition_prob
        candidate_scores[prev_label_idx] = score
    best_score = max(candidate_scores)
    best_prev_idx = candidate_scores.index(best_score)
    
    # Extract the path from the best last label using the backtrack matrix
    # Hint: the path contains the index of the best tag for each word
    # TODO
    cur_path = [best_prev_idx]
    for word_idx in reversed(range(len(sentence))):
        prev_link = cur_path[-1]
        new_link = backtrack[prev_link][word_idx]
        cur_path.append(new_link)
    
    # Reverse the path and convert the indexes to labels
    # TODO
    return [labels_list_exc_special[ind] for ind in reversed(cur_path)][1:]
viterbi(['this', 'is', 'a', 'very', 'good', 'chocolate', '.'])

* b) Ensure that the best path is saved during the decoding, so that you can extract the labels. What is the accuracy of your implementation of the Viterbi algorithm on the development data (`pos-data/en_ewt-dev.conll`)?

**Hint**: If implemented correctly, it should score at least an accuracy of 50%. If you score lower, we suggest you try printing the probabilities at each step (word) for the first sentence of the development data.

* c) **Bonus**: try to improve your predictions by inspecting common errors, tuning some of the decisions (e.g. smoothing, weighing the three probabilities) you made, or improving the handling of unknown tokens.


In [ ]:
cor = 0.0
total = 0
for sent in dev_data:
    preds = viterbi(sent[0])
    for gold, pred in zip(sent[1], preds):
        total += 1
        if gold == pred:
            cor += 1
print(cor/total)

# Lecture 10: BERT

## 3. Subword tokenization

BERT models are trained to predict tokens that were masked with a special `[mask]` token. In this assignment you will inspect what it has learned, and whether it has certain preferences (i.e. probing). 

a) Load the multilingual Bert tokenizer:

In [2]:
from transformers import AutoTokenizer

tokzr = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', use_fast=False)

Multilingual BERT was trained on the 100 most frequent languages of Wikipedia. They used smoothing, to correct inbalances in the data. However, their smoothing is relatively conservative, so high-resource languages have a higher impact on the model, and it is unclear how they sampled for training the tokenizer. Compare the tokenizations for two different language types you know; preferably one higher-resource and one lower-resource. If you only know 1 language, or only high-resource languages, try to use a different variety of the language (for example for English, use social media abbreviations or typos, e.g.: c u tmrw). Can you observe any differences in the results? does it match your intuition of separating mostly meaning-carrying subwords?

You can use Figure 1 of https://arxiv.org/pdf/1911.02116.pdf or https://en.wikipedia.org/wiki/List_of_Wikipedias to see how large languages are on Wikipedia.

In [12]:
# English vs Frisian
print(tokzr.tokenize('this is an example input'))
print(tokzr.tokenize('dit is in foarbyld input'))
# Exmaple is split into three words in Frisian, but remains a single word in English. 
# Morphologically, splitting it into two subwords (foar ##byld) could have made sense, 
# but the language is underrepresented.

# English vs Dutch
print(tokzr.tokenize('money lover'))
print(tokzr.tokenize('geld liefhebber'))
# Dutch has a long word, but also 2 times as many subwords, the split of ##he ##bber 
# would not have occured given a more specialized tokenizer

# English vs Danish
print(tokzr.tokenize('Yes exactly! Fortunately, I don\'t get them as gifts anymore'))
print(tokzr.tokenize('ja, præcis! Heldigvis får jeg dem ikke i gave mere.'))
# Even though they have the same length, the segmentation of præcis is not very 
# meaningful, of course due to the rare character. On the other hand gift-s vs gave 
# seems to make sense 

['this', 'is', 'an', 'example', 'input']
['dit', 'is', 'in', 'foar', '##byl', '##d', 'input']
['money', 'lover']
['geld', 'lief', '##he', '##bber']
['Yes', 'exactly', '!', 'Fortuna', '##tely', ',', 'I', 'don', "'", 't', 'get', 'them', 'as', 'gift', '##s', 'any', '##more']
['ja', ',', 'pr', '##æ', '##cis', '!', 'Held', '##ig', '##vis', 'får', 'jeg', 'dem', 'ikke', 'i', 'gave', 'mere', '.']


b) Test whether the `bert-base-cased` model can solve the analogy task that we discussed in the word2vec lecture ([slides](https://github.itu.dk/robv/intro-nlp2023/blob/main/slides/07-vector_semantics.pdf), [assignment](https://github.itu.dk/robv/intro-nlp2023/blob/main/assignments/week4/week4.ipynb)), we can do this by masking the target word we are looking for, and let the model predict which words fit best. We can then use a prompt to discover what the language model would guess. For example, we can use the prompt "man is to king as woman is to [MASK]". Try at least two syntactic analogies, and two semantic analogies.
You can use the following code:

(Note that you need 4gb of RAM for this assignment, otherwise you can use the HPC.)

In [18]:
from transformers import AutoModelForMaskedLM,AutoTokenizer
import torch

def getTopN(inputSent, model, tokzr, topn=1):
    maskId = tokzr.convert_tokens_to_ids(tokzr.mask_token)
    tokenIds = tokzr(inputSent).input_ids
    if maskId not in tokenIds:
        return 'please include ' + tokzr.mask_token + ' in your input'
    maskIndex = tokenIds.index(maskId)
    logits = model(torch.tensor([tokenIds])).logits
    return tokzr.convert_ids_to_tokens(torch.topk(logits, topn, dim=2).indices[0][maskIndex])

tokzr = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-cased')
getTopN('This is a [MASK] test.', model, tokzr, 5)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['positive', 'simple', 'negative', 'diagnostic', 'successful']

In [24]:
print(getTopN('work is to working as beg is to [MASK].', model, tokzr))
print(getTopN('houses is to house as cars is to [MASK].', model, tokzr))

print(getTopN('swim is to water as fly is to [MASK].', model, tokzr))
print(getTopN('Sjælland is to Denmark as Holland is to [MASK].', model, tokzr))
print(getTopN('Copenhagen is to Denmark as Amsterdam is to [MASK].', model, tokzr))
print(getTopN('Amsterdam is to The Netherlands as Copenhagen is to [MASK].', model, tokzr))

['work']
['house']
['fly']
['Norway']
['Germany']
['Denmark']


c) Test how robust the language model is, does it have an effect on the results of the word predictions if you include punctuations at the end of the sentence?, what about starting with a capital? and do typos have a large impact?

In [25]:
print(getTopN('amsterdam is to the netherlands as copenhagen is to [MASK].', model, tokzr))

print(getTopN('Amstrdam is too The Nethrlands as Copnhagen is to [MASK].', model, tokzr))

print(getTopN('Amsterdam is to The Netherlands as Københavns is to [MASK].', model, tokzr))

# It is pretty sensitive to any alternation

['Holland']
['be']
['Denmark']


d) Think of some prompts that test whether the model has any gender biases, you can test this for example by using common gendered names or pronouns, swapping them and then check whether the predicted word changed.

In [30]:
print(getTopN('Robs occupation is [MASK].', model, tokzr))
print(getTopN('Johns occupation is [MASK].', model, tokzr))
print(getTopN('Elisa\'s occupation is [MASK].', model, tokzr))

print(getTopN('Rob works as a [MASK].', model, tokzr))
print(getTopN('John works as a [MASK].', model, tokzr))
print(getTopN('Elisa works as a [MASK].', model, tokzr))

print(getTopN('[MASK] occupation is teacher.', model, tokzr))
print(getTopN('[MASK] occupation is nurse.', model, tokzr))
print(getTopN('[MASK] occupation is researcher.', model, tokzr))

# The small sample with names is not consistent enough to draw conclusions from, but the last three show a clear trend.

['farming']
['unknown']
['unknown']
['waiter']
['carpenter']
['nurse']
['His']
['Her']
['His']


# 4. Finetune a BERT model

We have provided code for training a BERT based classifier, which can be found in `week5/bert/bert-topic.py`. The implementation uses huggingface's transformers library (https://github.com/huggingface/transformers), and simply adds a linear layer to convert the output of the CLS token from the last layer of the masked language model to a label. 

a) Inspect the code; what should the shape of the output_scores be at the end of the forward pass?, What does this output represent?

b) Train the model on your own machine or on the HPC without a GPU (Note that this code needs ~8gb ram), how long does it take?

c) Now change the number of maximum training sentences (MAX_TRAIN_SENTS) to 500 and the batch size (BATCH_SIZE) to 32. Note that it will now take very long to train without a GPU. Train the model on the HPC, make sure you reserve a GPU to speed up the training. For more information, see http://hpc.itu.dk/scheduling/templates/gpu/ (only available on ITU network/VPN). Note that the code detects automatically whether a GPU is available. Also note that the transformers library is already installed, and can be loaded with:

```
module load PyTorch/1.7.1-foss-2020b
module load Transformers/4.2.1-foss-2020a-Python-3.8.2
``` 

(which you also have to put in the job script).

a) Batch size by the number of labels, in our case 8 by 4. This means that there are 4 scores for each instance in a batch, and each score represents the weight the model attributes to a certain class in label2id.